In [52]:
import os
print(os.getcwd())

work_directory_R = os.getcwd() + "/Results_ROC"
work_directory_C = os.getcwd() + "/Results_CM"
work_directory_P = os.getcwd() + "/ResultsPredicted"
Results_File = os.getcwd() + "/Metrics_Result.txt"

import shutil

## Try to remove tree; if failed show an error using try...except on screen
try:
    shutil.rmtree(work_directory_R)
    shutil.rmtree(work_directory_C)
    shutil.rmtree(work_directory_P)
    os.remove(Results_File)
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))
    
import gc as g

# %reset
from IPython import get_ipython
get_ipython().magic('reset -sf') 

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)


import tensorflow as tf,keras
from keras import backend as K, optimizers, regularizers
# K.clear_session()

import pickle, sys,os,cv2, numpy as np, pandas as pd, scipy, seaborn as sns, json, joblib
from random import shuffle
from keras.models import Sequential, Model, load_model, model_from_json
from keras.models import Model
from keras.layers import Lambda, LeakyReLU, multiply, Permute, Reshape, UpSampling2D
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, ZeroPadding2D, merge
from keras.layers.merge import concatenate
from keras import utils as np_utils
from keras.regularizers import l2

from keras.optimizers import Adam,RMSprop,SGD
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator,array_to_img
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, EarlyStopping

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.cm as cm

get_ipython().magic('matplotlib inline')
%matplotlib inline

from keras import layers
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix, roc_curve, auc, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm
from functools import partial
from sklearn import metrics
from collections import Counter
from itertools import cycle
from six.moves import cPickle as pickle
from scipy import interp
from skimage.feature import local_binary_pattern,hog
from skimage import data, exposure

import itertools
from time import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

print('Libraries Loaded')

tf.__version__
tf.test.is_built_with_cuda()

# GPU Check 1
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


D:\Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_TRAIN_Dart_LUNG\Code_TSNE\Final\Models Others
Error: D:\Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_TRAIN_Dart_LUNG\Code_TSNE\Final\Models Others/Results_ROC - The system cannot find the path specified.
Libraries Loaded


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12396499203184480922, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15108276634
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3105240546842923273
 physical_device_desc: "device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5"]

In [53]:
def load_pickle(data_file,q):
    print('\nTring to load pickle from %s' % data_file)
    with open(data_file, 'rb') as file:
        datasets = pickle.load(file)
        dataset = datasets['dataset']

    X_train = dataset['X_train']
    Y_train = dataset['Y_train']
    
    print('\nPickle Loaded Successfully!')
    
    del dataset

    if q==1:
        print('\nX_train shape:', X_train.shape)
        print('Y_train shape:', Y_train.shape)
    
    return X_train,Y_train

In [54]:
def save_pickle(x_train,y_train, data_file):
    print('\nTrying to save pickle to %s' % data_file)
    
    X_train = x_train
    Y_train = y_train
    del x_train,y_train
    
    # creating dictionary to store trian and test data
    datasets = {'dataset' : {'X_train': X_train,'Y_train': Y_train}}

    with open(data_file, 'wb') as file:
        pickle.dump(datasets, file, protocol=pickle.HIGHEST_PROTOCOL)
        del datasets # to free up memory.
        
    print('\nPickle Saved Successfully!')

In [55]:
def resize_data(img_height,img_width,X_train_R,Y_train_R):
    
    Images_ = []
    y_ = []    
        
    print('\n\n =============== Resizing Files =============== ')
    for i in range(X_train_R.shape[0]):
        image=X_train_R[i,:,:]
        image=cv2.resize(image,(img_height,img_width))
        Images_.append(image)
        y_.append(Y_train_R[i])

    Images_F = np.asarray(Images_) 
    Labels_F = np.asarray(y_)    
    del Images_, y_

    return (Images_F,Labels_F) 

In [56]:
option_D0a = 1

img_height,img_width = 224,224

if option_D0a==1:
    print("\n\n\n DataSet-1 \n")
    path  = "D://Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_TRAIN_ICIAR/"
    
    data_file = 'PICKLESNEW/DataSet_ICIAR_AE_Train_B.pickle'

    num_classes=2
    loss_a = 'binary_crossentropy'
    
    X_train_,Y_train_ = load_pickle(data_file,1)

    # To train you need to resize the patches of any size user want.
#     X_train_,Y_train_ = resize_data(img_height,img_width,X_train_,Y_train_)
#     print(X_train_.shape)
#     print(Y_train_.shape)
    
#     data_file_224 = 'PICKLESNEW/DataSet_ICIAR_AE_Train_B_224.pickle'   
#     save_pickle(X_train_, Y_train_, data_file_224)
    
    print("\n")
    unique, counts = np.unique(Y_train_, return_counts=True)
    abc= dict(zip(unique, counts))
    print(abc)
    for i in range(len(abc)):
        print('Class %d: %d' % (i, abc[i]))    




 DataSet-1 


Tring to load pickle from PICKLESNEW/DataSet_ICIAR_AE_Train_B.pickle

Pickle Loaded Successfully!

X_train shape: (32215, 224, 224, 3)
Y_train shape: (32215,)


{0: 4970, 1: 27245}
Class 0: 4970
Class 1: 27245


In [57]:
print('\nx_train shape:', X_train_.shape)
print('y_train shape:', Y_train_.shape)

print("\n Train Portion \n")
unique, counts = np.unique(Y_train_, return_counts=True)
abc= dict(zip(unique, counts))
print(abc)
for i in range(len(abc)):
    print('Class %d: %d' % (i, abc[i]))

index = range(5425,27215)
X_train_ = np.delete(X_train_, index, axis=0)
Y_train_ = np.delete(Y_train_, index)
print('\nx_train shape:', X_train_.shape)
print('y_train shape:', Y_train_.shape)

print("\n Train Portion \n")
unique, counts = np.unique(Y_train_, return_counts=True)
abc= dict(zip(unique, counts))
print(abc)
for i in range(len(abc)):
    print('Class %d: %d' % (i, abc[i]))



x_train shape: (32215, 224, 224, 3)
y_train shape: (32215,)

 Train Portion 

{0: 4970, 1: 27245}
Class 0: 4970
Class 1: 27245

x_train shape: (10425, 224, 224, 3)
y_train shape: (10425,)

 Train Portion 

{0: 4970, 1: 5455}
Class 0: 4970
Class 1: 5455


In [58]:
if option_D0a==1:
    size1 = 0.80    #D1
#     size1 = 0.90    #D2
    X_train, X_testA, Y_train, Y_testA = train_test_split(X_train_, Y_train_, train_size=size1, random_state=106,shuffle=True)
    print('Train data:', X_train.shape,', Train labels:', Y_train.shape)
    print('Test data:', X_testA.shape,', Test labels:', Y_testA.shape)
    del X_train_, Y_train_
    
    print("\n Train Portion \n")
    unique, counts = np.unique(Y_train, return_counts=True)
    abc= dict(zip(unique, counts))
    print(abc)
    for i in range(len(abc)):
        print('Class %d: %d' % (i, abc[i]))

    print("\n Test Portion \n")
    unique, counts = np.unique(Y_testA, return_counts=True)
    abc= dict(zip(unique, counts))
    print(abc)
    for i in range(len(abc)):
        print('Class %d: %d' % (i, abc[i]))    
        

Train data: (8340, 224, 224, 3) , Train labels: (8340,)
Test data: (2085, 224, 224, 3) , Test labels: (2085,)

 Train Portion 

{0: 3959, 1: 4381}
Class 0: 3959
Class 1: 4381

 Test Portion 

{0: 1011, 1: 1074}
Class 0: 1011
Class 1: 1074


In [59]:
X_traina, Y_traina = X_train,Y_train
print('Train data:', X_traina.shape,', Train labels:', Y_traina.shape)

X_testa, Y_testa = X_testA,Y_testA
print('Test data:', X_testa.shape,', Test labels:', Y_testa.shape)

X_testaB, Y_testaB = X_testA,Y_testA


Train data: (8340, 224, 224, 3) , Train labels: (8340,)
Test data: (2085, 224, 224, 3) , Test labels: (2085,)


In [60]:
def save_pickle_N(x_train, y_train, data_file):
    print('\nTrying to save pickle to %s' % data_file)
    
    X_train = x_train
    Y_train = y_train
    del x_train, y_train
    
    # creating dictionary to store trian and test data
    datasets = {'dataset' : {'X_train': X_train, 'Y_train': Y_train}}

    with open(data_file, 'wb') as file:
        joblib.dump(datasets, file)

        del datasets # to free up memory.
    print('\nPickle Saved Successfully!')
    
def load_pickle_N(data_file,q):

    print('\nTring to load pickle from %s' % data_file)
    with open(data_file, 'rb') as file:
        datasets = joblib.load(file)
        dataset = datasets['dataset']

    print('\nPickle Loaded Successfully!')

    X_train = dataset['X_train']
    Y_train = dataset['Y_train']
    del dataset

    if q==1:
        print('\nX_train shape:', X_train.shape)
        print('Y_train shape:', Y_train.shape)
    
    return X_train,Y_train    


In [61]:
print('\nTrain Features')

work_directory_Pickles = os.getcwd() + "/PICKLESNEW"

if option_D0a==1:
    data_file_1 = '/AE_Train_ICIAR_R_B_N_224.pickle'

data_file_R = work_directory_Pickles + data_file_1

if not (os.path.exists(data_file_R)):
    totalData = X_traina
    totalData_Lables = Y_traina

    X_traina = X_traina.astype('float32')/255. 
    
    print('\nRAW Features: ',X_traina.shape)
    print('RAW Features Labels: ',Y_traina.shape)
    save_pickle_N(X_traina, Y_traina, data_file_R)


Train Features


In [62]:
print('\nTest Features')

if option_D0a==1:
    data_file_1 = '/AE_Test_ICIAR_R_B_N_224.pickle'

data_file_R_T = work_directory_Pickles + data_file_1

if not (os.path.exists(data_file_R_T)):
    totalData = X_testa
    totalData_Lables = Y_testa

    X_testa = X_testa.astype('float32')/255. 

    print('\nRAW Features: ',X_testa.shape)
    print('RAW Features Labels: ',Y_testa.shape)
    save_pickle_N(X_testa,Y_testa,data_file_R_T)


Test Features


In [63]:
def plot_cm(tl, pl, option_D0a,normalize, work_directory_C):
    
    if option_D0a==1:
        classes = ['Non-Carcinoma' , 'Carcinoma'] # the same order as labels 0,1
#         classes = ['B', 'IS', 'IN', 'N'] # the same order as labels 0,1,2,3
    elif option_D0a==2:
        classes = ['A', 'L', 'M', 'P', 'S'] # the same order as labels 0,1,2,3
    
    classesNames = classes    
    confusion = confusion_matrix(tl, pl) #.astype(np.float)

    if normalize:
        print("Normalized confusion matrix")
        confusion = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis]
    else:
        print('Confusion matrix, without normalization')
        
    fig, ax = plt.subplots(figsize=(15,10))

    cm_sum = np.sum(confusion, axis=1, keepdims=True)
    cm_perc = confusion / cm_sum.astype(float) * 100
    annot = np.empty_like(confusion).astype(str)
    nrows, ncols = confusion.shape
    for i in range(nrows):
        for j in range(ncols):
            c = confusion[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%' % (p)
            elif c == 0:
                annot[i, j] = '%.1f%%' % (p)
            else:
                annot[i, j] = '%.1f%%' % (p)
                                
    cm = pd.DataFrame(confusion, index=classesNames, columns=classesNames)
    cm.index.name = 'True Label'
    cm.columns.name = 'Predicted Label'
    rot = 45

    ax.tick_params(axis="both", pad=14, labelsize=0, length = 0)

    res = sns.heatmap(cm, cmap= "Blues", annot=annot, fmt='', ax=ax, annot_kws={"size": 32}, cbar=False,
                  linewidths=0.1, linecolor='gray')

    res.set_xlabel(res.get_xlabel(), fontsize = 24, ha='center')
    res.set_ylabel(res.get_ylabel(), fontsize = 24, ha='center')

    res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 24, ha='center')
    res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 24, ha='center')

    plt.savefig(work_directory_C + '/Confusion_Matrix_D' + str(option_D0a), dpi=1000, bbox_inches="tight")
    plt.show()


In [64]:
K.clear_session()
import gc
gc.collect()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.95,allow_growth = True)
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False,gpu_options=gpu_options))
K.set_session(sess)

if option_D0a==1:
    Path_S = os.path.join(os.getcwd() + '/Weights_ICIAR/Binary_Balanced')

Epochs_C = 25
learning_rate = 10e-5
Optimizer_C=Adam(lr=learning_rate)

Batch_Size = 64

Path_S_1 = os.path.join(os.getcwd())


# Instantiate the cross validator
kfold_splits = 5
# kfold_splits = 3
monitor='val_acc'
# monitor='val_loss'
dropout=0.5

patience=5
min_lr=1e-5
verbose=1
factor=0.3
mode='max'
# mode='min'
pooling = 'avg'

print('\t\t\t\t\n\n ========================= MODEL ========================= ')
Epochs = 200
   
#D1
X_train_R,Y_train_R = load_pickle_N(data_file_R,2)

#D1
X_test_R_T,Y_test_R_T = load_pickle_N(data_file_R_T,2)


				

 ========================= MODEL ========================= 

Tring to load pickle from D:\Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_TRAIN_Dart_LUNG\Code_TSNE\Final\Models Others/PICKLESNEW/AE_Train_ICIAR_R_B_N_224.pickle

Pickle Loaded Successfully!

Tring to load pickle from D:\Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_TRAIN_Dart_LUNG\Code_TSNE\Final\Models Others/PICKLESNEW/AE_Test_ICIAR_R_B_N_224.pickle

Pickle Loaded Successfully!


In [68]:
###### ###### ###### Deep Learning Models ###### ###### ###### 
def Pooling(pooling,x):
    if pooling == 'avg':
        x = GlobalAveragePooling2D()(x)
    elif pooling == 'max':
        x = GlobalMaxPooling2D()(x)

    return x





###### ###### ###### BreastNet ###### ###### ###### 

from keras.layers import Concatenate, Layer, DepthwiseConv2D, AveragePooling2D, InputSpec, Add
from keras.utils import conv_utils

def cbam_block(cbam_feature, ratio=8):
    # Author: @kobiso (https://github.com/kobiso)

    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=8):
    
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature._keras_shape[channel_axis]
    
    shared_layer_one = Dense(channel//ratio,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    
    avg_pool = GlobalAveragePooling2D()(input_feature)    
    avg_pool = Reshape((1,1,channel))(avg_pool)
    
    assert avg_pool.shape[1:] == (1,1,channel)
    avg_pool = shared_layer_one(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel//ratio)
    avg_pool = shared_layer_two(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)
    
    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1,1,channel))(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)
    max_pool = shared_layer_one(max_pool)
    assert max_pool.shape[1:] == (1,1,channel//ratio)
    max_pool = shared_layer_two(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)
    
    cbam_feature = Add()([avg_pool,max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
    
    return multiply([input_feature, cbam_feature])


def spatial_attention(input_feature):
    kernel_size = 7
    
    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = Permute((2,3,1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature
    
    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat)	
    assert cbam_feature.shape[-1] == 1
    
    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
        
    return multiply([input_feature, cbam_feature])


def residual_block(y, nb_channels, _strides=(1, 1), _project_shortcut=False):
    # Author: @mjdietzx (https://gist.github.com/mjdietzx)

    shortcut = y

    y = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
    y = BatchNormalization()(y)
    y = LeakyReLU()(y)

    y = Conv2D(nb_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
    y = BatchNormalization()(y)

    if _project_shortcut or _strides != (1, 1):
        shortcut = Conv2D(nb_channels, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    y = Add()([shortcut, y])
    y = LeakyReLU()(y)

    return y


def BreastNet(input_W=img_width, input_H= img_height, n_classes=4, dropout_=0.1):

    """
    M. Togaçar, K.B. Özkurt, B. Ergen et al., BreastNet: A novel ˘
    convolutional neural network model through histopathological images for the diagnosis of breast
    cancer, Physica A (2019), doi: https://doi.org/10.1016/j.physa.2019.123592 .
    """    
    init = Input((input_W,input_H,3))

    init = Input((input_W,input_H,3))
    x = Conv2D(32, (3, 3), activation=None, padding='same')(init) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3), activation=None, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x1 = MaxPooling2D((2,2))(x)
    
    x = Conv2D(64, (3, 3), activation=None, padding='same')(x1)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = cbam_block(x)
    x = residual_block(x, 64)
    x2 = MaxPooling2D((2,2))(x)
    
    x = Conv2D(128, (3, 3), activation=None, padding='same')(x2)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = cbam_block(x)
    x = residual_block(x, 128)
    x3 = MaxPooling2D((2,2))(x)
        
    ginp1 = UpSampling2D(size=(2, 2), interpolation='bilinear')(x1)
    ginp2 = UpSampling2D(size=(4, 4), interpolation='bilinear')(x2)
    ginp3 = UpSampling2D(size=(8, 8), interpolation='bilinear')(x3)
    
    hypercolumn = concatenate([ginp1, ginp2, ginp3]) 
    gap = GlobalAveragePooling2D()(hypercolumn)

    x = Dense(256, activation=None)(gap)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_)(x)
    
    x = Dense(256, activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Dense(n_classes, activation="softmax", name="BreastNet")(x)
   
    model = Model(init, y)
    return model





###### ###### ###### LiverNet ###### ###### ###### 

class BilinearUpsampling(Layer):

    """
        Bilinear Upsampling Class
    """
    
    def __init__(self, upsampling=(2, 2), data_format=None, **kwargs):

        """
            Constructor of Bilinear-Upsampling
        """
        
        super(BilinearUpsampling, self).__init__(**kwargs)
        self.data_format = K.normalize_data_format(data_format)
        self.upsampling = conv_utils.normalize_tuple(upsampling, 2, 'size')
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        height = self.upsampling[0] * \
            input_shape[1] if input_shape[1] is not None else None
        width = self.upsampling[1] * \
            input_shape[2] if input_shape[2] is not None else None
        return (input_shape[0],
                height,
                width,
                input_shape[3])

    def call(self, inputs):
        return tf.image.resize_bilinear(inputs, (int(inputs.shape[1]*self.upsampling[0]),
                                                   int(inputs.shape[2]*self.upsampling[1])))

    def get_config(self):
        config = {'size': self.upsampling,
                  'data_format': self.data_format}
        base_config = super(BilinearUpsampling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def aspp(x,input_shape,out_stride):

    """
        ASPP Block
        
        Arguments:
        
            x: input feature map to the ASPP block
            input_shape: input shape of the feature map
            out_stride: the output stride
            
        Returns: 
            
            output feature map after processing
    """
    
    b0=Conv2D(256,(1,1),padding="same",use_bias=False)(x)
    b0=BatchNormalization()(b0)
    b0=Activation("relu")(b0)

    b1=DepthwiseConv2D((3,3),dilation_rate=(2,2),padding="same",use_bias=False)(x)
    b1=BatchNormalization()(b1)
    b1=Activation("relu")(b1)
    b1=Conv2D(256,(1,1),padding="same",use_bias=False)(b1)
    b1=BatchNormalization()(b1)
    b1=Activation("relu")(b1)

    b2=DepthwiseConv2D((3,3),dilation_rate=(3,3),padding="same",use_bias=False)(x)
    b2=BatchNormalization()(b2)
    b2=Activation("relu")(b2)
    b2=Conv2D(256,(1,1),padding="same",use_bias=False)(b2)
    b2=BatchNormalization()(b2)
    b2=Activation("relu")(b2)	

    b3=DepthwiseConv2D((3,3),dilation_rate=(4,4),padding="same",use_bias=False)(x)
    b3=BatchNormalization()(b3)
    b3=Activation("relu")(b3)
    b3=Conv2D(256,(1,1),padding="same",use_bias=False)(b3)
    b3=BatchNormalization()(b3)
    b3=Activation("relu")(b3)

    b5=DepthwiseConv2D((3,3),dilation_rate=(6,6),padding="same",use_bias=False)(x)
    b5=BatchNormalization()(b5)
    b5=Activation("relu")(b5)
    b5=Conv2D(256,(1,1),padding="same",use_bias=False)(b5)
    b5=BatchNormalization()(b5)
    b5=Activation("relu")(b5)

    # b5 = DepthwiseConv2D((3,3),dilation_rate=(6,6),padding="same",use_bias=False)(x)

    out_shape=int(input_shape[0]/out_stride)
    b4=AveragePooling2D(pool_size=(out_shape,out_shape))(x)
    b4=Conv2D(256,(1,1),padding="same",use_bias=False)(b4)
    b4=BatchNormalization()(b4)
    b4=Activation("relu")(b4)
    b4=BilinearUpsampling((out_shape,out_shape))(b4)

    x=Concatenate()([b4,b0,b1,b2,b3,b5])
    return x

def LiverNet(input_W=img_width, input_H= img_height, n_classes=4, dropout_=0.1):
    
    """
        Method to create the final model
    """
    
    init = Input((input_W,input_H,3))
    x = Conv2D(32, (3, 3), activation=None, padding='same')(init) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3), activation=None, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x1 = MaxPooling2D((2,2))(x)
    
    x = Conv2D(64, (3, 3), activation=None, padding='same')(x1)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = cbam_block(x)
    x = residual_block(x, 64)
    x2 = MaxPooling2D((2,2))(x)
    
    x = Conv2D(128, (3, 3), activation=None, padding='same')(x2)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = cbam_block(x)
    x = residual_block(x, 128)
    x3 = MaxPooling2D((2,2))(x)
    
    x1_aspp = aspp(x1,(112,112),1)
    x2_aspp = aspp(x2,(56,56),1)
    x3_aspp = aspp(x3,(28,28),1)

    ginp1 = UpSampling2D(size=(2, 2), interpolation='bilinear')(x1_aspp)
    ginp2 = UpSampling2D(size=(4, 4), interpolation='bilinear')(x2_aspp)
    ginp3 = UpSampling2D(size=(8, 8), interpolation='bilinear')(x3_aspp)
    
    hypercolumn = Concatenate()([ginp1, ginp2, ginp3]) 
    gap = GlobalAveragePooling2D()(hypercolumn)

    x = Dense(256, activation=None)(gap)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_)(x)
    
    x = Dense(256, activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Dense(n_classes, activation="softmax", name="LiverNet")(x)
   
    model = Model(init, y)
    return model





###### ###### ###### HCNNet ###### ###### ###### 

# Function for a inception module block
def inception_module(layer_in, f1, f2, f3):
    # 1x1 conv
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
    # 3x3 conv
    conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(layer_in)
    # 5x5 conv
    conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(layer_in)
    # 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
    # concatenate filters, assumes filters/channels last
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return layer_out

def HCNNet(input_W=img_width, input_H= img_height, n_classes=4, dropout_=0.1):
    
    # define model input
    init = Input((input_W,input_H,3))
    x = Conv2D(64, (3, 3), activation=None, padding='same')(init) 
    x = Activation('relu')(x)
    x = MaxPooling2D((3,3), strides=(1,1), padding='same')(x)

    x = Conv2D(64, (3, 3), activation=None, padding='same')(x) 
    x = Activation('relu')(x)
    x = MaxPooling2D((3,3), strides=(1,1), padding='same')(x)

    # add inception module
    layer1 = inception_module(x, 64, 128, 32)
    layer2 = inception_module(layer1, 128, 192, 96)
    layer3 = inception_module(layer2, 192, 208, 48)
    layer4 = inception_module(layer3, 160, 224, 64)
    layer5 = inception_module(layer4, 128, 256, 64)
    layer6 = inception_module(layer5, 112, 288, 64)
    layer7 = inception_module(layer6, 256, 320, 128)

    x_ = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer7)

    layer8 = inception_module(x_, 256, 320, 128)
    layer9 = inception_module(layer8, 384, 384, 128)

    x_1 = MaxPooling2D((7,7), padding='same')(layer9)

    x_2 = Dense(1024, activation='relu')(Flatten()(x_1))

    y = Dense(n_classes, activation="softmax", name="HCNNet")(x_2)

    model = Model(init, y)
    return model





###### ###### ###### RuneCNN ###### ###### ###### 

def encoder(input_img):
    #Encoder
    x = Conv2D(8, (3, 3), strides=1, activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), strides=1,  padding='same')(x)

    x = Dropout(0.1)(x)
    x = Conv2D(8, (3, 3), strides=2, activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=1, padding='same')(x)

    x = Dropout(0.1)(x)
    x = Conv2D(8, (3, 3), strides=2, activation='relu', padding='same')(x)

    x = Dense(4096, activation='relu')(Flatten()(x))
    x = Dropout(0.1)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.1)(x)
    encoded = Dense(1024, activation='relu')(x)
#     print("shape of encoded", K.int_shape(encoded))

    return encoded

def classifier_C(encode,num_classes,dropout_):
    layer_1 = Dense(256, activation='relu')(encode)
    layer_2 = Dense(512, activation='relu')(layer_1)
    
    layer_1 = Dropout(dropout_)(layer_1)
    output = Dense(num_classes, activation='softmax', name="RuneCNN")(layer_2)
    return output

def RuneCNN(input_W=img_width, input_H= img_height, n_classes=4, dropout_=0.1):
    
    """
        Method to create the final model
    """
    
    input_img = Input((input_W,input_H,3))
    encode = encoder(input_img)
    model = Model(input_img,classifier_C(encode,n_classes,dropout_))

    return model

In [69]:
from keras.applications import vgg16, inception_v3, resnet50, mobilenet, densenet

print('\nSelect Model Option \n')
option_DM = int(input("\n 1-ResNet \n 2-DensNet \n 3-Inception \n 4-MobileNet \n 5-RuneCNN  \n 6-BreastNet \n 7-LiverNet  \n 8-HCNNet \n"))
while option_DM not in (1,2,3,4,5,6,7,8):
    option_DM = int(input("\n 1-ResNet \n 2-DensNet \n 3-Inception \n 4-MobileNet \n 5-RuneCNN  \n 6-BreastNet \n 7-LiverNet  \n 8-HCNNet \n"))

with tf.device("/device:GPU:0"):
    print('\nLoading Model \n')
    if option_DM==1:
        model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape= (img_height,img_width,3),classes=num_classes)
        filepath = "weights.best 5FOLD ResNet.hdf5"

        # ResNet
        x = Pooling(pooling,model.output)
        x = Dense(1024, activation="relu", kernel_regularizer=l2(0.0001))(x)
        x = Dropout(dropout)(x)
        x = Dense(512, activation="relu", kernel_regularizer=l2(0.0001))(x)
        x = Dropout(dropout)(x)
        Last_FC_Layer = Dense(num_classes, activation= 'softmax', name='Last_Output_Layer')(x)
        
    elif option_DM==2:
        model = densenet.DenseNet201(weights='imagenet', include_top=False, input_shape= (img_height,img_width,3),classes=num_classes)
        filepath = "weights.best 5FOLD DenseNet.hdf5"

        # DensNet
        x = Pooling(pooling,model.output)
        x = Dense(1024, activation="relu", kernel_regularizer=l2(0.0001))(x)
        x = Dense(512, activation="relu", kernel_regularizer=l2(0.0001))(x)
        x = Dropout(dropout)(x)
        Last_FC_Layer = Dense(num_classes, activation= 'softmax', name='Last_Output_Layer')(x)
        
    elif option_DM==3:
        model = inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape= (img_height,img_width,3),classes=num_classes)
        filepath = "weights.best 5FOLD Inception.hdf5"

        # InceptionNet
        x = Pooling(pooling,model.output)
        x = Dense(1024, activation="relu", kernel_regularizer=l2(0.0001))(x)
        x = Dense(512, activation="relu", kernel_regularizer=l2(0.0001))(x)
        x = Dropout(dropout)(x)
        Last_FC_Layer = Dense(num_classes, activation= 'softmax', name='Last_Output_Layer')(x)

    elif option_DM==4:
        model = mobilenet.MobileNet(weights='imagenet', include_top=False, input_shape= (img_height,img_width,3),classes=num_classes)
        filepath = "weights.best 5FOLD MobileNet.hdf5"

        # MobileNet
        x = Pooling(pooling,model.output)
        x = Dense(1024, activation="relu", kernel_regularizer=l2(0.0001))(x)
        x = Dense(512, activation="relu", kernel_regularizer=l2(0.0001))(x)
        x = Dropout(dropout)(x)
        Last_FC_Layer = Dense(num_classes, activation= 'softmax', name='Last_Output_Layer')(x)
        
    elif option_DM==5:
        # RuneCNN
        gpu_model = RuneCNN(img_width, img_height, num_classes, dropout)
        filepath = "weights.best 5FOLD RuneCNN.hdf5"

    elif option_DM==6:
        # BreastNet
        gpu_model = BreastNet(img_width, img_height, num_classes, dropout)
        filepath = "weights.best 5FOLD BreastNet.hdf5"

    elif option_DM==7:
        # LiverNet
        gpu_model = LiverNet(img_width, img_height, num_classes, dropout)
        filepath = "weights.best 5FOLD LiverNet.hdf5"

    elif option_DM==8:
        gpu_model = HCNNet(img_width, img_height, num_classes, dropout)
        filepath = "weights.best 5FOLD HCNNet.hdf5"
        
#     print('\nModel Loaded \n')

if option_DM==1 or option_DM==2 or option_DM==3 or option_DM==4:
    gpu_model = Model(inputs= model.input, outputs= Last_FC_Layer)
    
gpu_model.compile(loss=loss_a,optimizer=Optimizer_C,metrics=['accuracy'])



Select Model Option 


 1-ResNet 
 2-DensNet 
 3-Inception 
 4-MobileNet 
 5-RuneCNN  
 6-BreastNet 
 7-LiverNet  
 8-HCNNet 
7

Loading Model 



In [70]:
# ORIGINAL ONE
print('\n\n =============== Classifier Parameters =============== ')
print('\nLoss: ', loss_a)
print('Optimizer: ', Optimizer_C)
print('Batch Size: ', Batch_Size)
print('Epoch: ', Epochs_C)

print('\n\n =============== Train Data Dimensions =============== ')

X_trainS,Y_trainS = X_train_R, Y_train_R
print('\nFeatures R: ',X_train_R.shape)
print('Feature Labels: ',Y_trainS.shape)

print('\n\n =============== Test Data Dimensions =============== ')
X_testS,Y_testS = X_test_R_T, Y_test_R_T
print('\nFeatures R: ',X_test_R_T.shape)





 =============== Classifier Parameters =============== 

Loss:  binary_crossentropy
Optimizer:  <keras.optimizers.Adam object at 0x000001651E3B45C0>
Batch Size:  64
Epoch:  25


 =============== Train Data Dimensions =============== 

Features R:  (8340, 224, 224, 3)
Feature Labels:  (8340,)


 =============== Test Data Dimensions =============== 

Features R:  (2085, 224, 224, 3)


In [71]:
print('\n\n\t\t =============== Model Summary =============== ')
print(gpu_model.summary())




		 =============== Model Summary =============== 
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 224, 224, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 224, 224, 32) 128         conv2d_4[0][0]                   
__________________________________________________________________________________________________
activation_4 (Activation)       (None, 224, 224, 32) 0           batch_normalization_4[0][0]      
_________________________________________________________

In [73]:
model_history = []  
cv_scores = []  

skf = StratifiedKFold(n_splits=kfold_splits, shuffle=True)

# Learning Rate Reducer
learn_control = ReduceLROnPlateau(monitor=monitor, patience= patience,verbose=verbose,factor=factor, min_lr=min_lr)

if not (os.path.exists(filepath)):
    checkpoint = ModelCheckpoint(filepath, monitor=monitor, verbose=verbose, save_best_only=True, mode=mode)
    print('\nNot Using real-time data augmentation.')
    print('Training Starts...')      
    print('X_train Concatenate shape:', X_trainS.shape)
    print('Y_train Concatenate shape:', Y_trainS.shape)                                                                                                                                                                      

    for index, (train_indices, val_indices) in enumerate(skf.split(X_trainS,Y_trainS)):
        print("\n\n\tTraining on fold " + str(index+1) + "/" + str(kfold_splits) + "...")
        xtrain_R, xvalid_R = X_trainS[train_indices], X_trainS[val_indices]
        ytrain_R, yvalid_R = Y_trainS[train_indices], Y_trainS[val_indices]    

        ytrain = tf.keras.utils.to_categorical(ytrain_R, num_classes)
        yvalid = tf.keras.utils.to_categorical(yvalid_R, num_classes)        

        print('\nAfter Applying Categorical Opeartion')
        print('Train data:', xtrain_R.shape,', Train labels:', ytrain.shape)
        print('Validation data:', xvalid_R.shape,', Validation labels:', yvalid.shape)

#                     print("Mode: ", Mode)
        with tf.device('/device:GPU:0'):
            history = gpu_model.fit(xtrain_R, ytrain, batch_size=Batch_Size, epochs=Epochs_C, validation_data=(xvalid_R, yvalid), verbose=1, callbacks=[learn_control, checkpoint])
        model_history.append(history)    
        print('\nEvaluating the Model...')
        scores = gpu_model.evaluate(xvalid_R, yvalid,verbose=0)
        print("%s: %.2f%%" % (gpu_model.metrics_names[1], scores[1]*100))
        cv_scores.append(scores[1] * 100)

        del xtrain_R, xvalid_R, ytrain, yvalid, ytrain_R, yvalid_R     

    print("\n%s: %.2f%%" % ("Mean Accuracy: ",np.mean(cv_scores)))
    print("%s: %.2f%%" % ("Standard Deviation: +/-", np.std(cv_scores)))
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cv_scores), np.std(cv_scores)))
    print('Estimated Accuracy %.3f (%.3f)' % (np.mean(cv_scores), np.std(cv_scores)))

else:
# load the models
    print('\n Classifier Loading Saved Models')
    print('Model : ',filepath)
    gpu_model = load_model(filepath)

print("\n")                
Y_testa_Old = Y_testS
print('Before Categorical Features : ',Y_testa_Old.shape)

Y_testS_ = tf.keras.utils.to_categorical(Y_testS, num_classes)      
print('After Categorical Features : ',Y_testS_.shape)

test_eval = gpu_model.evaluate(X_test_R_T,Y_testS_, verbose=0)

print('\n\n =============== Overall Average Results =============== ')
print('\nAccuracy:', test_eval[1])

#Predicted Labels
predicted_classes = gpu_model.predict(X_test_R_T)
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)
test_labels = Y_testS

print("Sensitivity = ", recall_score(test_labels, predicted_classes, average='weighted'))
print("Precision = ", precision_score(test_labels, predicted_classes, average='weighted'))
print("F1 = ", f1_score(test_labels, predicted_classes, average='weighted'))


print('\n\n =============== Classwise Results =============== ')
cmat = confusion_matrix(test_labels, predicted_classes) 
print("\nAccuracy = ", cmat.diagonal()/cmat.sum(axis=1))
print("Sensitivity = ", recall_score(test_labels, predicted_classes, average=None))
print("Precision = ", precision_score(test_labels, predicted_classes, average=None))
print("F1 = ", f1_score(test_labels, predicted_classes, average=None))


print("\n\n")            
target_names = ["Class {}".format(ia) for ia in range(num_classes)]
print(classification_report(test_labels, predicted_classes, target_names=target_names))


with open(Path_S_1 + "/Metrics_Result Others ICAIR.txt", "a") as myfile:

    myfile.write("\n\n =============== Dataset_" + str(option_D0a) + "=============== ")
#     myfile.write("\nAE Configuration: No. of Layers = " + str(Number_of_Hidden_Layers_Input) + ", No. of Neurons = " + str(Number_of_Neurons_Input) + ", Activation No = " + str(Activation_Function_Input) + "\n")                    
    myfile.write("\n =============== Overall Average Results =============== ")
    myfile.write("\nAccuracy: " + str("%.5f" % test_eval[1]))
    myfile.write("\nLoss: " + str("%.5f" % test_eval[0]))
    myfile.write("\nSensitivity: " + str(recall_score(test_labels, predicted_classes, average='weighted')))
    myfile.write("\nPrecision: " + str(precision_score(test_labels, predicted_classes, average='weighted')))
    myfile.write("\nF-1_Score: " + str(f1_score(test_labels, predicted_classes, average='weighted')) + "\n")

    myfile.write("\n =============== Classwise Average Results =============== ")
    myfile.write("\nAccuracy: " + str(cmat.diagonal()/cmat.sum(axis=1)))
    myfile.write("\nSensitivity: " + str(recall_score(test_labels, predicted_classes, average=None)))
    myfile.write("\nPrecision: " + str(precision_score(test_labels, predicted_classes, average=None)))
    myfile.write("\nF-1_Score: " + str(f1_score(test_labels, predicted_classes, average=None)) + "\n")

columns = 16
work_directory_P = os.getcwd() + "/ResultsPredicted"

if not os.path.exists(work_directory_P):
    os.mkdir(work_directory_P)

fig=plt.figure(figsize=(8, 8))

left  = 0.125  # the left side of the subplots of the figure
right = 1    # the right side of the subplots of the figure
bottom = 0.1   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.25   # the amount of width reserved for blank space between subplots
hspace = 0.25   # the amount of height reserved for white space between subplots

plt.subplots_adjust( left, bottom, right, top, wspace, hspace)
columns = 5
rows = 5
f = 12
print('\n\n =============== Correct Predicted Samples =============== ')

correct = np.where(predicted_classes==test_labels)[0]
print("\nFound %d correct labels" % len(correct))
for i1, correct in enumerate(correct[:rows*columns]):
    ax = fig.add_subplot(rows, columns, i1+1)
    plt.imshow(X_testaB[correct].astype('uint8'))
    plt.axis('off')        
    ax.set_title("Predict {}, Class {}".format(predicted_classes[correct], test_labels[correct]), fontsize=f)
    plt.imsave(work_directory_P+'/Correc_D-'+ str(option_D0a) + '_PL-' + str(predicted_classes[correct]) + '_TL-' + str(test_labels[correct]) + '_I-' + str(i1) + '.jpg', X_testaB[correct].astype('uint8'), dpi=fig.dpi)
plt.tight_layout()
plt.show()


fig=plt.figure(figsize=(8, 8))
plt.subplots_adjust( left, bottom, right, top, wspace, hspace)

print('\n\n =============== Incorrect Predicted Samples =============== ')

incorrect = np.where(predicted_classes!=test_labels)[0]
print("\nFound %d incorrect labels" % len(incorrect))
for i2, incorrect in enumerate(incorrect[:rows*columns]):
    ax = fig.add_subplot(rows, columns, i2+1)
    plt.imshow(X_testaB[incorrect].astype('uint8'))
    plt.axis('off')        
    ax.set_title("Predict {}, Class {}".format(predicted_classes[incorrect], test_labels[incorrect]), fontsize=f)
    plt.imsave(work_directory_P+'/Incorrect_D-'+ str(option_D0a) + '_PL-' + str(predicted_classes[incorrect]) + '_TL-' + str(test_labels[incorrect]) + '_I-' + str(i2) + '.jpg', X_testaB[incorrect].astype('uint8'), dpi=fig.dpi)
plt.tight_layout()
plt.show()

print('\n\n ===============  Results =============== ')

y_score = gpu_model.predict([X_test_R_T,X_test_H_T,X_test_L_T])
y_test = tf.keras.utils.to_categorical(Y_testS, num_classes)      
print(y_score.shape)
print(y_test.shape)

# Plot linewidth.
lw = 1

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for ia in range(num_classes):
    fpr[ia], tpr[ia], _ = roc_curve(y_test[:, ia], y_score[:, ia])
    roc_auc[ia] = auc(fpr[ia], tpr[ia])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area
all_fpr = np.unique(np.concatenate([fpr[ib] for ib in range(num_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for ic in range(num_classes):
    mean_tpr += interp(all_fpr, fpr[ic], tpr[ic])

# Finally average it and compute AUC
mean_tpr /= num_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(1)
plt.style.use('default')    
plt.rcParams.update({'font.size': 13})

plt.plot(fpr["micro"], tpr["micro"],
         label='Micro Average (Area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=3)

plt.plot(fpr["macro"], tpr["macro"],
         label='Macro Average (Area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=3)


prop_cycle = plt.rcParams['axes.prop_cycle']
colors = cycle(prop_cycle.by_key()['color'])
for id, color in zip(range(num_classes), colors):
    if id==0:
        plt.plot(fpr[id], tpr[id], color=next(colors), lw=lw,
                 label='Class {0} (Area = {1:0.2f})'
                 ''.format('A', roc_auc[id]))
    elif id==1:
        plt.plot(fpr[id], tpr[id], color=next(colors), lw=lw,
                 label='Class {0} (Area = {1:0.2f})'
                 ''.format('L', roc_auc[id]))
    elif id==2:
        plt.plot(fpr[id], tpr[id], color=next(colors), lw=lw,
                 label='Class {0} (Area = {1:0.2f})'
                 ''.format('M', roc_auc[id]))
    elif id==3:
        plt.plot(fpr[id], tpr[id], color=next(colors), lw=lw,
                 label='Class {0} (Area = {1:0.2f})'
                 ''.format('P', roc_auc[id]))
    elif id==4:
        plt.plot(fpr[id], tpr[id], color=next(colors), lw=lw,
                 label='Class {0} (Area = {1:0.2f})'
                 ''.format('S', roc_auc[id]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.10, 1.10])
plt.ylim([-0.10, 1.10])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='upper right', bbox_to_anchor=(1.75, 1.0))

#             work_directory_R = os.getcwd() + "/Results_ROC"
#             work_directory_C = os.getcwd() + "/Results_CM"

#             if not os.path.exists(work_directory_R):
#                 os.mkdir(work_directory_R)

#             plt.savefig(work_directory_R + '/ROC_D' + str(option_D0a), dpi=1000, bbox_inches="tight")
#             plt.show()

#             if not os.path.exists(work_directory_C):
#                 os.mkdir(work_directory_C)

#             normalize = False
#             plot_cm(test_labels, predicted_classes, option_D0a, normalize, work_directory_C)

del gpu_model, predicted_classes, test_labels
# ORIGINAL CODE
